In [1]:
!pip install av
!pip install ultralytics
!pip install streamlit

In [7]:
# Python In-built packages
from pathlib import Path
import PIL
from ultralytics import YOLO
# External packages
import streamlit as st

In [3]:
import os
import sys
import shutil
HOME = '/content'
os.makedirs(HOME + '/project2', exist_ok = True)

%cd {HOME}/project2

/content/project2


In [4]:
os.makedirs(os.getcwd() + '/images',exist_ok = True)
os.makedirs(os.getcwd() + '/videos',exist_ok = True)

In [56]:
shutil.copy( "/content/drive/MyDrive/yolov8m/training_results/inventory_train2/weights/best.pt", '/content/project2')
shutil.copy( "/content/drive/MyDrive/connector/trim.mp4", '/content/project2/videos')
shutil.copy( "/content/drive/MyDrive/connector/helper.py", '/content/project2')
shutil.copy( "/content/drive/MyDrive/connector/settings.py", '/content/project2')
shutil.copy( "/content/drive/MyDrive/connector/IMG20230413104630.jpg", '/content/project2/images')

'/content/project2/images/IMG20230413104630.jpg'

In [6]:
%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
%cd {HOME}/ByteTrack

# workaround related to https://github.com/roboflow/notebooks/issues/80
!sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

!pip3 install -q -r requirements.txt
!python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric
# workaround related to https://github.com/roboflow/notebooks/issues/112 and https://github.com/roboflow/notebooks/issues/106
!pip install -q loguru lap thop

from IPython import display
display.clear_output()


import sys
sys.path.append(f"{HOME}/ByteTrack")


import yolox
print("yolox.__version__:", yolox.__version__)

yolox.__version__: 0.1.0


In [8]:
# Install the THOP package for FLOPs and model size computation
!pip install thop

# Install the YOLOX package for object detection
!pip install yolox

# Install the Supervision package for computer vision tasks
!pip install supervision==0.1.0

In [9]:
# change directory to Project2
%cd {HOME}/project2

/content/project2


In [19]:
%%writefile app.py

# Python In-built packages
from pathlib import Path
import PIL

# External packages
import streamlit as st
import numpy as np

# Local Modules
import settings
import helper
import tempfile
# Setting page layout
st.set_page_config(
    page_title="TMT Steel Rods Track and Count",
    page_icon="ðŸ¤–",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Main page heading
st.title("TMT Steel Rods Detection")


# Load Pre-trained ML Model
model_path = settings.MODEL

try:
    model = helper.load_model(model_path)
except Exception as ex:
    st.error(f"Unable to load model. Check the specified path: {model_path}")
    st.error(ex)

st.sidebar.header("Image/Video Config")
source_radio = st.sidebar.radio(
    "Select Source", settings.SOURCES_LIST)

source_img = None
# If image is selected
if source_radio == settings.IMAGE:
    source_img = st.sidebar.file_uploader(
        "Choose an image...", type=("jpg", "jpeg", "png", 'bmp', 'webp'))

    col1, col2 = st.columns(2)

    with col1:
        try:
            if source_img is None:
                default_image_path = str(settings.DEFAULT_IMAGE)
                default_image = PIL.Image.open(default_image_path)
                st.image(default_image_path, caption="Default Image",
                         use_column_width=True)
            else:
                uploaded_image = PIL.Image.open(source_img)
                st.image(source_img, caption="Uploaded Image",
                         use_column_width=True)
        except Exception as ex:
            st.error("Error occurred while opening the image.")
            st.error(ex)

    with col2:
        if source_img is None:
            default_image_path = str(settings.DEFAULT_IMAGE)
            default_image = PIL.Image.open(default_image_path)
            helper.initialize_class_variables()
            helper.initialize_tracker()
            res = helper.display_detected_frames(model, default_image)

            # Plot the detected objects on the video frame
            st.image(res, caption='Detected Image', channels="BGR", use_column_width=True)
        else:
            if st.sidebar.button('Detect Objects'):
                uploaded_image = PIL.Image.open(source_img)
                helper.initialize_class_variables()
                helper.initialize_tracker()
                res = helper.display_detected_frames(model, uploaded_image)
                st.image(res, caption='Detected Image', use_column_width=True)

elif source_radio == settings.VIDEO:
    source_video = st.sidebar.file_uploader(
        "Choose a video...", type=("mp4", "MJPG", "mp4v"))
    try:
        if source_video is None:
            default_video_path = str(settings.VIDEO_1_PATH)
            helper.initialize_class_variables()
            helper.initialize_tracker()
            helper.play_stored_video(default_video_path, model)
        else:
            # Check if a video file was uploaded
            if source_video is not None:
                # Create a temporary file
                temp_file = tempfile.NamedTemporaryFile(delete=False)

                # Save the uploaded video in the temporary file
                temp_file.write(source_video.read())

                # Close the temporary file
                temp_file.close()

                # Get the file path of the temporary file
                video_path = temp_file.name
                if st.sidebar.button('Detect Objects'):
                  helper.initialize_class_variables()
                  helper.initialize_tracker()
                  helper.play_stored_video(video_path, model)
            else:
                st.warning("Please upload a video file.")
    except Exception as ex:
        st.error("Error occurred while opening the image.")
        st.error(ex)
else:
    st.error("Please select a valid source type!")

Overwriting app.py


In [11]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/project2/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/project2/package.json'
npm WARN project2 No description
npm WARN project2 No repository field.
npm WARN project2 No README data
npm WARN project2 No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.465s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [57]:
!streamlit run app.py &>/content/logs.txt & curl ipv4.icanhazip.com

35.187.228.233


In [58]:
!npx localtunnel --port 8501

npx: installed 22 in 1.6s
your url is: https://soft-games-grin.loca.lt
^C
